Neural networks, especially complex neural networks, are resource-hungry algorithms. When it comes to training neural networks on medium to large datasets, the amount of computation time to adequately train a model can take hours (or even days!) With simple modelling problems, like the ones covered in this module, training a model in the same notebook as an analysis is no problem. However, with more formal applications of neural network and deep learning models, data scientists cannot afford the time or resources to build and train a model each time they analyze data. In these cases, a trained model must be stored and accessed outside of the training environment.

With TensorFlow, we have the ability to save and load neural network models at any stage, including partially trained models. When building a TensorFlow model, if we use Keras' ModelCheckpoint method, we can save the model weights after it tests a set number of data points. Then, at any point, we can reload the checkpoint weights and resume model training. Saving checkpoints while training has a number of benefits:

+ We can short-circuit our training loop at any time (stop the function by pressing CTRL+C, or by pressing the stop button at the top of the notebook). This can be helpful if the model is showing signs of overfitting.
+ The model is protected from computer problems (power failure, computer crash, etc.). Worst-case scenario: We would lose five epochs' worth of optimization.
+ We can restore previous model weight coefficients to try and revert overfitting.

Let's practice generating checkpoint files and loading model weights from different epochs. To make things simple, we'll implement checkpoints to our previous deep learning example notebook. To start, we'll open our "DeepLearning_Tabular" notebook and rerun all of the code in the notebook for the following steps:

+ Import dependencies.
+ Import the input dataset.
+ Generate categorical variable list.
+ Create a OneHotEncoder instance.
+ Fit and transform the OneHotEncoder.
+ Add the encoded variable names to the DataFrame.
+ Merge one-hot encoded features and drop the originals.
+ Split the preprocessed data into features and target arrays.
+ Split the preprocessed data into training and testing dataset.
+ Create a StandardScaler instance.
+ Fit the StandardScaler.
+ Scale the data.
+ Define the model.
+ Add first and second hidden layers.
+ Add the output layer.
+ Check the structure of the model.

In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
attrition_df = pd.read_csv('HR-Employee-Attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


Looking at the top of our DataFrame, there are multiple columns with categorical values as well as our numerical values. To make things easier, we should generate a list of categorical variable names. Instead of searching across all 35 columns and keeping track of which variables need categorical preprocessing, we'll let Python do all of the heavy lifting. As an added bonus, we can use our variable list to perform the one-hot encoding once, rather than for each individual variable. To generate our variable list, we'll use Pandas Dataframe.dtypes property. Add and run the following code in your notebook:

In [2]:
# Generate our categorical variable list
attrition_cat = attrition_df.dtypes[attrition_df.dtypes == "object"].index.tolist()
attrition_cat

['Attrition',
 'BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'Over18',
 'OverTime']

Now that we have our variable names separated, we can start to preprocess our data starting with the one-hot encoding of the categorical data. Looking at our attrition_cat variable, there are eight categorical variables that need encoding. However, before we loop through our variables and encode them using Scikit-learn's OneHotEncoder module, we need to make sure that none of the categorical variables have more than 10 unique values and require bucketing. To check the three variables, add and run the following code:

In [3]:
# Check the number of unique values in each column
attrition_df[attrition_cat].nunique()

Attrition         2
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
dtype: int64

According to the nunique method, none of the categorical variables have more than 10 unique values, which means we're ready to encode. Previously, we have encoded a single variable at a time, but Scikit-learn is flexible enough to perform all of the one-hot encodings at the same time. The only difference from our single variable example is we need to pass our attrition_cat variable list. Again, add and run the following code to your notebook:

In [4]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(attrition_df[attrition_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names_out(attrition_cat)
encode_df.head()

,Attrition_No,Attrition_Yes,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


Now that our categorical variables have been encoded, they are ready to replace our unencoded categorical variables in our dataset.

To replace these columns, we'll use a combination of Pandas' merge and drop methods. Add and run the following code in the notebook:

In [5]:
# Merge one-hot encoded features and drop the originals
attrition_df = attrition_df.merge(encode_df, left_index=True, right_index=True)
attrition_df = attrition_df.drop(attrition_cat,1)
attrition_df.head()

C:\Users\elleg\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1102,1,2,1,1,2,94,3,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,49,279,8,1,1,2,3,61,2,2,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,37,1373,2,2,1,4,4,92,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,33,1392,3,4,1,5,4,56,3,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,27,591,2,1,1,7,1,40,3,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


Next we must split our training and testing data, then standardize our numerical variables using Scikit-learn's StandardScaler module.

REWIND
We need to split our training and testing data before fitting our StandardScaler instance. This prevents testing data from influencing the standardization function.

To build our training and testing datasets, we need to separate two values:

+ input values (which are our independent variables commonly referred to as model features or "X" in TensorFlow documentation)
+ target output (our dependent variable commonly referred to as target or "y" in TensorFlow documentation)

For our purposes, we want to build a model that will predict whether or not a person is at risk for attrition; therefore, we must separate the "Attrition" columns from the rest of the input data. In fact, because the attrition data is dichotomous (one of two values), we only need to keep the "Attrition_Yes" column—we can ignore the "Attrition_No" column because it is redundant. To separate our features and target as well as perform our training/test split, add and run the following code in your notebook:

In [6]:
# Split our preprocessed data into our features and target arrays
y = attrition_df["Attrition_Yes"].values
X = attrition_df.drop(["Attrition_Yes", "Attrition_No"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

C:\Users\elleg\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


Now that our training and testing data have been allocated, we're ready to build our StandardScalerobject and standardize the numerical features. Add and run the following code to the notebook:

In [7]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Now that our data is preprocessed via one-hot encoding and standardization, we should probably perform a gut check to ensure that no data has been lost from our original DataFrame.

At last, our data is preprocessed and separated and ready for modelling. For our purposes, we will use the same framework we used for our basic neural network:

+ For our input layer, we must add the number of input features equal to the number of variables in our feature DataFrame.
+ In our hidden layers, our deep learning model structure will be slightly different—we'll add two hidden layers with only a few neurons in each layer. To create the second hidden layer, we'll add another Keras Dense class while defining our model. All of our hidden layers will use the relu activation function to identify nonlinear characteristics from the input values.
+ In the output layer, we'll use the same parameters from our basic neural network including the sigmoid activation function. The sigmoid activation function will help us predict the probability that an employee is at risk for attrition.

To create our deep learning model, we must add and run the following code:

In [8]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 448       
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


Now that we have our training data and our model defined, we're ready to compile and train our model using checkpoints. To use checkpoints, we need to define the checkpoint file name and directory path. For our purposes, we'll label our checkpoints by epoch number and contain them within their own folder. This ensures that our checkpoint files are neat, organized, and easily identifiable. Add and run the following code to our notebook:

In [12]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

Once we have defined the file structure and filepath, we need to create a callback object for our deep learning model. A callback object is used in the Keras module to define a set of functions that will be applied at specific stages of the training process. There are a number of different callback functions available that can create log files, force training to stop, send training status messages, or in our case save model checkpoints. To create an effective checkpoint callback using the ModelCheckpoint method, we need to provide the following parameters:

+ filepath=checkpoint_path—the checkpoint directory and file structure we defined previously
+ verbose=1—we'll be notified when a checkpoint is being saved to the directory
+ save_weights_only=True—saving the full model each time can fill up a hard drive very quickly; this ensures that the checkpoint files take up minimal space
+ save_freq='epoch'—checkpoints will be saved every epoch

Bringing it all together, we can compile, train, and evaluate our deep learning model by adding and running the following code:

In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
34/35 [============================>.] - ETA: 0s - loss: 0.5416 - accuracy: 0.8254 
Epoch 1: saving model to checkpoints\weights.01.hdf5
35/35 [==============================] - 1s 3ms/step - loss: 0.5408 - accuracy: 0.8267
Epoch 2/100
 1/35 [..............................] - ETA: 0s - loss: 0.5039 - accuracy: 0.8750
Epoch 2: saving model to checkpoints\weights.02.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.5183 - accuracy: 0.8267
Epoch 3/100
 1/35 [..............................] - ETA: 0s - loss: 0.4845 - accuracy: 0.8438
Epoch 3: saving model to checkpoints\weights.03.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.4956 - accuracy: 0.8285
Epoch 4/100
 1/35 [..............................] - ETA: 0s - loss: 0.5153 - accuracy: 0.7500
Epoch 4: saving model to checkpoints\weights.04.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.4682 - accuracy: 0.8330
Epoch 5/100
 1/35 [..............................] - ETA: 0

Epoch 36/100
 1/35 [..............................] - ETA: 0s - loss: 0.1950 - accuracy: 0.9062
Epoch 36: saving model to checkpoints\weights.36.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.2273 - accuracy: 0.9093
Epoch 37/100
 1/35 [..............................] - ETA: 0s - loss: 0.2953 - accuracy: 0.8438
Epoch 37: saving model to checkpoints\weights.37.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.2257 - accuracy: 0.9083
Epoch 38/100
 1/35 [..............................] - ETA: 0s - loss: 0.2521 - accuracy: 0.8438
Epoch 38: saving model to checkpoints\weights.38.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.2241 - accuracy: 0.9102
Epoch 39/100
 1/35 [..............................] - ETA: 0s - loss: 0.3008 - accuracy: 0.8750
Epoch 39: saving model to checkpoints\weights.39.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.2227 - accuracy: 0.9120
Epoch 40/100
 1/35 [..............................] 

35/35 [==============================] - 0s 2ms/step - loss: 0.1686 - accuracy: 0.9419
Epoch 71/100
 1/35 [..............................] - ETA: 0s - loss: 0.0988 - accuracy: 0.9375
Epoch 71: saving model to checkpoints\weights.71.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.1678 - accuracy: 0.9419
Epoch 72/100
 1/35 [..............................] - ETA: 0s - loss: 0.0764 - accuracy: 1.0000
Epoch 72: saving model to checkpoints\weights.72.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.1661 - accuracy: 0.9410
Epoch 73/100
 1/35 [..............................] - ETA: 0s - loss: 0.0951 - accuracy: 0.9375
Epoch 73: saving model to checkpoints\weights.73.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.1653 - accuracy: 0.9383
Epoch 74/100
 1/35 [..............................] - ETA: 0s - loss: 0.1780 - accuracy: 0.9062
Epoch 74: saving model to checkpoints\weights.74.hdf5
35/35 [==============================] - 0s 1ms/step

After running the previous code, we have created our trained model within the Python session, as well as a folder of checkpoints we can use to restore previous model weights. Now if we ever need to restore weights, we can use the Keras Sequential model's load_weights method to restore the model weights. To test this functionality, let's define another deep learning model, but restore the weights using the checkpoints rather than training the model. Once again we must add and run the following to our notebooks:

In [14]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.4033 - accuracy: 0.8777 - 192ms/epoch - 16ms/step
Loss: 0.4033026397228241, Accuracy: 0.8777173757553101


Using the checkpoints, we were able to regenerate the model instantaneously and confirm the model was able to produce the exact same results.

Checkpoints are a great way to save model weights during training, but they fall short when it comes to sharing a trained model. In data science, trained models are published in scientific papers, deployed in software, open-sourced on GitHub, not to mention passed along to colleagues. In these cases, it is not practical to pass along only model weights, which can cause frustration and confusion. Instead, we can use the Keras Sequential model's save method to export the entire model (weights, structure, and configuration settings) to an Hierarchical Data Format (HDF5Links to an external site.) file. Once saved, anyone can import the exact same trained model to their environment by using the Keras load_model method and use it for analysis.

NOTE
Even though we can save full neural network and deep learning models using Keras checkpoints, each full model file is almost ten times the size of a weight-only file. For those with limited hard drive space, saving full models using checkpoints is not feasible.

To practice exporting and importing our entire model, we'll use the same notebook as our previous section.

Currently, in our notebook environment, we should have a fully trained classification model that can predict employee attrition based on features in the dataset. To export the trained model, we need to add and run the following code:

In [15]:
# Export our model to HDF5 file
nn_new.save("trained_attrition.h5")

After running the code, we should see a file named "trained_attrition.h5," which contains the complete model and configuration. Now that we have the model saved, we can create the model at any point. Let's try importing the model into the notebook without providing any structure or context. To import the model, add and run the following code:

In [16]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('trained_attrition.h5')

Lastly, we can test the performance of the completed model on our test dataset by adding and running the following code:

In [17]:
# Evaluate the completed model using the test data
model_loss, model_accuracy = nn_imported.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.4033 - accuracy: 0.8777 - 122ms/epoch - 10ms/step
Loss: 0.4033026397228241, Accuracy: 0.8777173757553101


Looking at the final results, our imported model was able to reproduce the exact same performance metrics as the original model. Using this same procedure, we can import any type of Keras model for evaluation on a dataset with the same features.